In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import sys
from pathlib import Path
import anndata as ad
import scanpy as sc
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns
from matplotlib import pyplot as plt

sys.path.insert(0, "/home/ylu/project")
from utils import *
import time

%load_ext autoreload
%autoreload 2

/home/ylu/anaconda3/envs/SPACEL/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import SPACEL
from SPACEL import Scube, Splane

In [3]:
order = [
    155, 153,151,149,147,144,142,140, 138, 143,141,139, 137,135,133,130,131,
129,127,125,123,121,119,117,115,113,111,109,107,105,103, 101, 99, 97, 95, 93,
91,89,87,85,81,83,79,77,75,73,71,67,65,63,61,59,57,55,53,51,49,47,45,43,41,39,
37,33,31,29,27,25,28,30,32,34,36,38,40,42,44,48,50,52,54,56,58,60,62,
    64,66,68,72,74,76,78,80,82,84,88,90,92,94,96,98,100,102,104,106,108,
    110,112,114,116,118,120,122,124,136,134,132,128,126
]
data_folder = "/dm7100f/yifan/StereoSeq_macaque_processed"
files = [f'macaque_T{o}.h5ad' for o in order]

In [4]:
# data_folder = "./results/split_data/"
results_folder = "./results/SPACEL/"
cache_folder = os.path.join(results_folder, 'Scube_outputs')
figures_folder = "./results/figures/SPACEL"
Path(results_folder).mkdir(parents=True, exist_ok=True)
Path(cache_folder).mkdir(parents=True, exist_ok=True)

In [5]:
## Perform the Spateo alignment
from tqdm import tqdm
import time
sampling_num = 20000
spatial_key = 'r_spatial'
key_added = 'align_spatial'
anno_key = 'SubClass'
for i in tqdm(range(len(files)-1)):
    slice1 = ad.read_h5ad(os.path.join(data_folder, files[i]))
    slice2 = ad.read_h5ad(os.path.join(data_folder, files[i+1]))
    slice1.obsm[spatial_key] = slice1.obsm[spatial_key].astype(np.float64)
    slice2.obsm[spatial_key] = slice2.obsm[spatial_key].astype(np.float64)
    inliers1 = np.isnan(slice1.obsm[spatial_key].sum(1)) | (slice1.obs[anno_key].values.astype(str) == 'nan')
    inliers2 = np.isnan(slice2.obsm[spatial_key].sum(1)) | (slice2.obs[anno_key].values.astype(str) == 'nan')
    slice1 = slice1[~inliers1]
    slice2 = slice2[~inliers2]
    sampline_idx1 = np.random.choice(slice1.shape[0], sampling_num, replace=False) if slice1.shape[0] > sampling_num else np.arange(slice1.shape[0])
    sampline_idx2 = np.random.choice(slice2.shape[0], sampling_num, replace=False) if slice2.shape[0] > sampling_num else np.arange(slice2.shape[0])
    slice1 = slice1[sampline_idx1,:]
    slice2 = slice2[sampline_idx2,:]

    slice1.obsm['spatial_2D'] = slice1.obsm[spatial_key].copy()
    slice2.obsm['spatial_2D'] = slice2.obsm[spatial_key].copy()
    slice1.obsm['spatial'] = slice1.obsm[spatial_key].copy()
    slice2.obsm['spatial'] = slice2.obsm[spatial_key].copy()
    align_slices = [slice1, slice2]
    time_start = time.time()
    Scube.align(
        align_slices,
        cluster_key=anno_key, 
        n_neighbors = 15, 
        n_threads=10,
        p=2,
        write_loc_path=os.path.join(results_folder, 'Scube_outputs/aligned_coordinates.csv')
    )
    align_slices[0].obsm[key_added] = align_slices[0].obsm['spatial_aligned'].values
    align_slices[1].obsm[key_added] = align_slices[1].obsm['spatial_aligned'].values
    # align_slices[1].uns['align_spatial_iter'] = morpho_model.iter_added
    time_end = time.time()
    R1, t1=solve_RT_by_correspondence(align_slices[1].obsm[key_added], align_slices[1].obsm[spatial_key])
    R2, t2=solve_RT_by_correspondence(align_slices[0].obsm[spatial_key], align_slices[0].obsm[key_added])
    t = t1 @ R2.T + t2
    R = R2 @ R1
    alignment_results = {'R': R, 't': t, 'time': time_end - time_start}
    np.save(os.path.join(results_folder, f"slice_{i}_{i+1}_sampling_{sampling_num}.npy"), alignment_results, allow_pickle=True)

  0%|                                     | 0/41 [00:00<?, ?it/s]

Start alignment...
Alignment slice 1 to 0
Runtime: 212.70057034492493 s


  2%|▋                         | 1/41 [03:45<2:30:28, 225.71s/it]

Start alignment...
Alignment slice 1 to 0
Runtime: 213.38423466682434 s


  5%|█▎                        | 2/41 [07:34<2:27:44, 227.31s/it]

...

 95%|███████████████████████▊ | 39/41 [2:33:02<06:15, 187.53s/it]

Start alignment...
Alignment slice 1 to 0
Runtime: 182.0645558834076 s


 98%|████████████████████████▍| 40/41 [2:36:08<03:07, 187.29s/it]

Start alignment...
Alignment slice 1 to 0
Runtime: 196.85711240768433 s


100%|█████████████████████████| 41/41 [2:39:27<00:00, 233.35s/it]


In [ ]:
## Perform the SPACEL with provided annotation alignment
sampling_num = 20000
for i in tqdm(range(len(slices)-1)):
    slice1, slice2 = slices[i].copy(), slices[i+1].copy()
    sampline_idx1 = np.random.choice(slice1.shape[0], sampling_num, replace=False) if slice1.shape[0] > sampling_num else np.arange(slice1.shape[0])
    sampline_idx2 = np.random.choice(slice2.shape[0], sampling_num, replace=False) if slice2.shape[0] > sampling_num else np.arange(slice2.shape[0])
    slice1 = slice1[sampline_idx1,:]
    slice2 = slice2[sampline_idx2,:]
    slice1.obsm['spatial_2D'] = slice1.obsm[spatial_key]
    slice2.obsm['spatial_2D'] = slice2.obsm[spatial_key]
    slice1.obsm['spatial'] = slice1.obsm[spatial_key]
    slice2.obsm['spatial'] = slice2.obsm[spatial_key]
    align_slices = [slice1, slice2]
    time_start = time.time()
    Scube.align(
        align_slices,
        cluster_key=anno_key, 
        n_neighbors = 15, 
        n_threads=10,
        p=2,
        write_loc_path=os.path.join(results_folder, 'Scube_outputs/aligned_coordinates.csv')
    )
    align_slices[0].obsm[key_added] = align_slices[0].obsm['spatial_aligned'].values
    align_slices[1].obsm[key_added] = align_slices[1].obsm['spatial_aligned'].values
    time_end = time.time()
    
    R1, t1=solve_RT_by_correspondence(align_slices[1].obsm[key_added], align_slices[1].obsm[spatial_key])
    R2, t2=solve_RT_by_correspondence(align_slices[0].obsm[spatial_key], align_slices[0].obsm[key_added])
    t = t1 @ R2.T + t2
    R = R2 @ R1
    alignment_results = {'R': R, 't': t, 'time': time_end - time_start}
    np.save(os.path.join(results_folder, f"slice_{i}_{i+1}_sampling_{sampling_num}.npy"), alignment_results, allow_pickle=True)

  0%|                                                                                                                                                                                                          | 0/18 [00:00<?, ?it/s]

Start alignment...
Alignment slice 1 to 0
Runtime: 238.14924597740173 s


  6%|██████████▌                                                                                                                                                                                    | 1/18 [03:59<1:07:55, 239.75s/it]

Start alignment...
Alignment slice 1 to 0
Runtime: 241.6222219467163 s


 11%|█████████████████████▏                                                                                                                                                                         | 2/18 [08:03<1:04:33, 242.12s/it]

Start alignment...
Alignment slice 1 to 0
